In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "../src"))

In [8]:
import logging
import jax
import jax.numpy as np
import numpy as onp
import skbio.stats.composition as cmp
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import statsmodels.api as sm
from datetime import datetime

from simulate_data_fct import sim_IV_lognormal_linear
from plot_fct import update_layout, img_path, update_layout_px, colours, plot_ilr_X_vs_Y
from run_methods_all import run_methods_confidence_interval
from plot_fct import plot_mse_results, plot_beta_results

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# set up logging file
logging.getLogger().setLevel(logging.INFO)

In [5]:
key = jax.random.PRNGKey(191)

# Linear Setup

## Data Generation

In [15]:
n = 500
p = 30
num_star = 250
instrument_strength = "unknown"
savepath = os.path.join(img_path, "HighDimLogcontrast")

V = cmp._gram_schmidt_basis(p)
num_inst=10
c_X = 2  # dispersion parameter

# instrument strength
alpha0 =np.hstack([np.array([1, 1, 2, 1, 4, 4, 2, 1, 4, 4, 2, 1]), jax.random.choice(key, np.array([1, 2, 2]), (p-8,))])
alphaT = jax.random.choice(key, np.array([0, 0, 0, 10]), (num_inst, p))
# confounder
mu_c = np.hstack([np.array([0.2, 0.3, 0.2, 0.1]), jax.random.uniform(key, (p-4, ), minval=0.01, maxval=0.05)])
mu_c = mu_c / mu_c.sum()  # has to be a compositional vector
ps = np.hstack([np.zeros((int(p/2),)), 0.8*np.ones((p - int(p/2),))])   # prob of zero inflation

# relationship between X and Y
beta0 = 1
betaT = np.hstack([ np.array([-5, -5, -5, -5]), np.array([5, 5, 5, 5]),  
                   np.zeros((p-8))])  # beta is chosen to sum up to one
# confounder influence to Y
c_Y = np.hstack([np.array([20, 20, 0, 0]), np.array([-5, -5, -5, -5, -5, -5, -5, -5]), 
                 np.zeros((p-12))])  # confounder is a composition as well.

In [10]:
confounder, Z_sim, X_sim, Y_sim, X_star, Y_star = sim_IV_lognormal_linear(
            key,
            n=n,
            p=p,
            num_inst=num_inst,
            mu_c=mu_c,
            c_X=c_X,
            alpha0=alpha0,
            alphaT=alphaT,
            c_Y=c_Y,
            beta0=beta0,
            betaT=betaT,
            num_star=500,
ps=ps)
X_sim_ilr = cmp.ilr(X_sim)
X_star_ilr = cmp.ilr(X_star)

## Data Visualization

In [11]:
# Compute F-Statistics 
ZZ_sim = onp.array(sm.add_constant(Z_sim))
alphahat = np.linalg.inv(ZZ_sim.T@ZZ_sim)@ZZ_sim.T@X_sim_ilr
MSM = np.sum(((ZZ_sim@alphahat - np.mean(X_sim_ilr, axis=0))**2), axis=0)/(num_inst)
MSE = np.sum(((ZZ_sim@alphahat - X_sim_ilr)**2), axis=0)/(n-num_inst)
F = MSM / MSE
F

DeviceArray([21.7243    ,  6.557183  ,  5.0914493 ,  2.2304416 ,
              4.7095346 ,  0.967368  ,  2.3969598 ,  5.781007  ,
             14.037651  ,  7.829588  ,  5.8779984 , 14.1642    ,
              3.339064  , 17.386314  ,  1.3577465 ,  1.2088323 ,
              1.5491704 ,  0.45316425,  1.0237327 ,  1.5300292 ,
              1.7684637 ,  1.1605805 ,  1.5496781 ,  0.66334194,
              0.70978135,  2.1112206 ,  0.45974594,  0.6181574 ,
              2.2108054 ], dtype=float32)

In [16]:
fig = plot_ilr_X_vs_Y(X_sim_ilr, Y_sim, Y_star)

fig.update_layout(xaxis3=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(yaxis3=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(xaxis4=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(yaxis4=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(xaxis5=dict(showline=True, linewidth=2, linecolor="black"))
fig.update_layout(yaxis5=dict(showline=True, linewidth=2, linecolor="black"))

fig.write_image(os.path.join(savepath,
                             "Data_vs_trueeffect_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))


fig.show()

# Diversity Approach

In [17]:
from plot_fct import plot_diversity_methods
from helper_fct import diversity
from run_methods_all import run_diversity_estimation_methods
div_shannon = diversity(X_sim, "shannon")
div_simpson = diversity(X_sim, "simpson")

In [18]:
# Shannon Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_shannon, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

fig = plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(savepath,
                             "ShannonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

INFO:absl:Setup matrices...
INFO:absl:Optimize lambda...
INFO:absl:Optimal lambda [3.98270133]...
INFO:absl:Optimize xi...


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.287
Date:                 Di, 01 Jun 2021   Prob (F-statistic):              0.235
Time:                        16:56:48   Log-Likelihood:                -702.98
No. Observations:                 500   AIC:                             1428.
Df Residuals:                     489   BIC:                             1474.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0265      0.305      0.087      0.9

INFO:absl:Optimal xi [6.91830488]...
INFO:absl:Predict treatment effect...


In [19]:
# Simpson Estimation
x, y, ytrue, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results = run_diversity_estimation_methods(
    Z_sim, div_simpson, Y_sim, Ytrue=Y_star, methods=["OLS", "2SLS", "KIV"])

plot_diversity_methods(x, y, xstar, xstar_bound, ystar_ols, ystar_2sls, ystar_kiv, results, ytrue)
fig.write_image(os.path.join(savepath,
                             "SimpsonDiversity_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()

INFO:absl:Setup matrices...
INFO:absl:Optimize lambda...
INFO:absl:Optimal lambda [3.91361359]...
INFO:absl:Optimize xi...


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.296
Date:                 Di, 01 Jun 2021   Prob (F-statistic):              0.229
Time:                        16:56:49   Log-Likelihood:                -702.93
No. Observations:                 500   AIC:                             1428.
Df Residuals:                     489   BIC:                             1474.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0384      0.305     -0.126      0.9

INFO:absl:Optimal xi [6.65859303]...
INFO:absl:Predict treatment effect...


# Confidence Intervals for higher dimensional methods

In [20]:
from run_methods_all import run_methods_all

In [ ]:
today = datetime.now()
num_iteration = 20

path = os.getcwd()
mypath = os.path.join(path, "temp",
                      today.strftime("%Y_%m_%d_%H_%M") + "_Logcontrast_ConfidenceInterval_"+str(instrument_strength)
                      +"_"+ str(num_iteration)+"_Microbiota_" +str(p))

if not os.path.isdir(mypath):
    os.makedirs(mypath)


# create logging file with relevant information
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    filename=os.path.join(os.getcwd(), mypath, "configurations.log"),
                    filemode='w'
                    )

logging.info("n="+str(n)+", p="+str(p)+", instrument strength: "+instrument_strength)
logging.info("Number of runs: "+str(num_iteration))
logging.info("First Stage Specification: ")
logging.info("Number of Instruments: "+str(num_inst))
logging.info("Instrument Strength by F-Test: " +str(F))
logging.info("alpha0="+str(alpha0)+", alphaT="+str(alphaT)+", confounder multiplicator c_X="+str(c_X))
logging.info("Second Stage Specification: ")
logging.info("beta0="+str(beta0)+", betaT="+str(betaT)+", confounder multiplicator c_Y="+str(c_Y))
logging.info("Confounder mean mu_c="+str(mu_c))


df_beta, df_mse, mse_large_confidence= run_methods_confidence_interval(
    key, num_iteration, 
    n, p, num_inst, mu_c, c_X, alpha0, alphaT, c_Y, beta0, betaT, is_lognormal=True, num_star=num_star, 
    logcontrast_threshold=0.3)

**************************************************************************************************
*****************************************We are at 0 of 20***********************************************
**************************************************************************************************


/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/skbio/stats/composition.py:464: RuntimeWarning:

divide by zero encountered in log

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/skbio/stats/composition.py:466: RuntimeWarning:

invalid value encountered in subtract



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [-5. -5. -5. -5.  5.  5.  5.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Estimated Beta: [ 0.76  2.23  4.71 -2.63 -3.74  2.16 -1.84  3.7   5.66  2.19  0.18  3.52
  6.25 -0.95  2.13 -0.73  4.07  2.38 -1.02  2.41  0.39  5.13 -3.55  1.51
  2.72  3.11  5.23  2.46  7.15]
Estimated Beta: [ 4.98  3.9   1.71 -1.91  5.11  6.86  1.11  5.08 -0.56 -3.04  0.03  1.93
 -1.57 -4.67  2.34 -0.78  2.04 -2.86 -1.36  2.01 -1.46  0.5  -4.37  4.28
 -0.74 -2.03 -2.53 -4.8  -2.17 -7.03]
Error: 4202.0

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [-5. -5. -5. -5.  5.  5.  5.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/base/model.py:1832: ValueWarning:

covariance of constraints does not have full rank. The number of constraints is 29, but rank is 19

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1787: RuntimeWarning:

invalid value encountered in sqrt



                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                     -37.499
Model:                         IV2SLS   Adj. R-squared:                -39.875
Method:                     Two Stage   F-statistic:                 1.374e-16
                        Least Squares   Prob (F-statistic):               1.00
Date:                 Di, 01 Jun 2021                                         
Time:                        18:46:15                                         
No. Observations:                 500                                         
Df Residuals:                     470                                         
Df Model:                          29                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -39.8759   3.38e+09  -1.18e-08      1.0

...

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/skbio/stats/composition.py:464: RuntimeWarning:

divide by zero encountered in log

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/skbio/stats/composition.py:466: RuntimeWarning:

invalid value encountered in subtract



---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - ILR Regression >>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [-5. -5. -5. -5.  5.  5.  5.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Estimated Beta: [ 1.34  1.82  2.6  -1.82 -1.49  3.12 -1.35  2.99  4.51  0.79  2.27  2.22
  5.55 -0.37  3.54 -0.02  3.33  1.95 -1.    1.78  1.48  4.42 -2.36  0.35
  1.68  3.1   3.5   1.6   7.52]
Estimated Beta: [ 5.09  3.19  1.9   0.4   4.68  4.69 -0.04  4.29 -0.15 -2.08  1.37 -0.25
 -0.39 -4.02  1.71 -2.29  1.15 -2.3  -1.07  1.86 -0.94 -0.72 -3.79  2.94
  0.28 -1.09 -2.6  -3.12 -1.32 -7.39]
Error: 3318.24

---------------------------------------------------------------------------------------------
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ONLY Second Stage - Log Contrast >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
True Beta: [-5. -5. -5. -5.  5.  5.  5.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/base/model.py:1832: ValueWarning:

covariance of constraints does not have full rank. The number of constraints is 29, but rank is 19

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1787: RuntimeWarning:

invalid value encountered in sqrt



                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                      -0.971
Model:                         IV2SLS   Adj. R-squared:                 -1.093
Method:                     Two Stage   F-statistic:                 5.170e-15
                        Least Squares   Prob (F-statistic):               1.00
Date:                 Di, 01 Jun 2021                                         
Time:                        19:11:52                                         
No. Observations:                 500                                         
Df Residuals:                     470                                         
Df Model:                          29                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.1278        nan        nan        n

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/skbio/stats/composition.py:464: RuntimeWarning:

divide by zero encountered in log

/opt/anaconda3/envs/gen_iv_models/lib/python3.7/site-packages/skbio/stats/composition.py:466: RuntimeWarning:

invalid value encountered in subtract



In [30]:
import pickle
results = {
    "df_mse": df_mse,
    "df_beta": df_beta,
    "mse_large": mse_large_confidence
}

with open(os.path.join(savepath, "results_"+str(p)+str(instrument_strength)+".pickle"), "wb") as f:
    pickle.dump(results, f)
    f.close()


# Mean Squarred Error Plot

In [36]:
filter_list = ["ALR+LC", "ILR+ILR", "ONLY Second ILR", "ONLY Second LC", "DIR+LC", "ILR+LC"]

fig = plot_mse_results(df_mse, filter_list)
fig.update_yaxes(range=(0, 50))
fig.write_image(os.path.join(savepath,
                             "MSEFull_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+"_3.pdf" ))
fig.show()

# Beta Plot

In [39]:
fig = plot_beta_results(df_beta, V.T@betaT, filter_list)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_yaxes(range=(-15, 20))
fig.write_image(os.path.join(savepath,
                             "BetaFull_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()


In [43]:
fig = plot_beta_results(df_beta, V.T@betaT, filter_list, beta_zero=True)


fig.update_yaxes(range=(-5, 5))
fig.write_image(os.path.join(savepath,
                             "BetaZeroFull_" +str(p)+"InstrumentStrength_"+str(instrument_strength)+".pdf" ))

fig.show()
